In [23]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import re
import np_utils
import seaborn as sns
import matplotlib.pyplot as plt
import string, os
import nltk
import re
import keras
import random
import io
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
import sys
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings("ignore")
import pickle

In [24]:
df = pd.read_csv("filtered_track_df.csv")

In [25]:
df.head()

,artists_id,acousticness,danceability,disc_number,duration_ms,energy,href,id,instrumentalness,key,...,speechiness,tempo,time_signature,track_href,uri,valence,release_date,artists_name,genres,release_year
0,68WwJXWrpo1yVOOIZjLSeT,0.0268,0.506,1.0,248777.0,0.741,https://api.spotify.com/v1/tracks/0UATU9OJxh4m...,0UATU9OJxh4m3fwDljdGZn,0.000027,1.0,...,0.0349,94.042,4.0,https://api.spotify.com/v1/tracks/0UATU9OJxh4m...,0UATU9OJxh4m3fwDljdGZn,0.236,2018-09-28,Evalyn,"['electropop', 'indie electro-pop', 'indie pop...",2018
1,09xj0S68Y1OU1vHMCZAIvz,0.5050,0.487,1.0,171573.0,0.297,https://api.spotify.com/v1/tracks/4JH1M62gVDND...,4JH1M62gVDNDhDAUiQB3Qv,0.000052,11.0,...,0.0915,185.912,3.0,https://api.spotify.com/v1/tracks/4JH1M62gVDND...,4JH1M62gVDNDhDAUiQB3Qv,0.289,2001-08-21,Café Tacvba,"['latin', 'latin alternative', 'latin rock', '...",2001
2,6pSsE5y0uJMwYj83KrPyf9,0.1330,0.629,1.0,207396.0,0.706,https://api.spotify.com/v1/tracks/0h7Ld5CvgzaU...,0h7Ld5CvgzaUN1zA3tdyPq,0.000000,1.0,...,0.4360,81.220,4.0,https://api.spotify.com/v1/tracks/0h7Ld5CvgzaU...,0h7Ld5CvgzaUN1zA3tdyPq,0.543,2019-01-25,Dawn Richard,"['alternative r&b', 'deep pop r&b', 'escape ro...",2019
3,7slfeZO9LsJbWgpkIoXBUJ,0.4060,0.590,1.0,279000.0,0.597,https://api.spotify.com/v1/tracks/4S1bYWrLOC8s...,4S1bYWrLOC8smuy8kJzxKQ,0.000023,9.0,...,0.0275,121.051,4.0,https://api.spotify.com/v1/tracks/4S1bYWrLOC8s...,4S1bYWrLOC8smuy8kJzxKQ,0.466,1995-09-12,Ricky Martin,"['dance pop', 'latin', 'latin pop', 'mexican p...",1995
4,09hVIj6vWgoCDtT03h8ZCa,0.0316,0.727,1.0,218773.0,0.380,https://api.spotify.com/v1/tracks/758mQT4zzlvB...,758mQT4zzlvBhy9PvNePwC,0.000000,7.0,...,0.3350,92.050,4.0,https://api.spotify.com/v1/tracks/758mQT4zzlvB...,758mQT4zzlvBhy9PvNePwC,0.455,1991-09-24,A Tribe Called Quest,"['alternative hip hop', 'conscious hip hop', '...",1991


In [26]:
df.shape

(9460, 28)

In [27]:
df.columns

Index(['artists_id', 'acousticness', 'danceability', 'disc_number', 'duration_ms', 'energy',
       'href', 'id', 'instrumentalness', 'key', 'liveness', 'loudness', 'lyrics', 'mode', 'name',
       'playlist', 'popularity', 'preview_url', 'speechiness', 'tempo', 'time_signature',
       'track_href', 'uri', 'valence', 'release_date', 'artists_name', 'genres', 'release_year'],
      dtype='object')

In [28]:
len(pd.unique(df["genres"]))

1418

In [29]:
def load_data():
    df = pd.read_csv("filtered_track_df.csv")
    df['genres'] = df.genres.apply(lambda x: [i[1:-1] for i in str(x)[1:-1].split(", ")])
    exploded_track_df = df.explode("genres")
    return exploded_track_df

In [30]:
exploded_track_df = load_data()

In [31]:
genre_names = ['dance pop', 'electronic', 'electropop', 'hip hop', 'jazz', 'k-pop', 'latin', 'pop', 'pop rap', 'r&b', 'rock']
exploded_track_df = exploded_track_df.explode("genres")[exploded_track_df.explode("genres")["genres"].isin(genre_names)]

In [32]:
pd.unique(exploded_track_df['genres'])

array(['electropop', 'latin', 'r&b', 'dance pop', 'pop', 'hip hop',
       'pop rap', 'rock', 'electronic', 'k-pop', 'jazz'], dtype=object)

In [33]:
exploded_track_df.groupby(['genres'])['id'].count()

genres
dance pop     2519
electronic     412
electropop    1022
hip hop        666
jazz            76
k-pop          608
latin         1977
pop           3499
pop rap       1072
r&b           1260
rock          1703
Name: id, dtype: int64

In [34]:
elec = exploded_track_df[exploded_track_df["genres"] == "electronic"]
#jazz["lyrics"] = jazz.lyrics.apply(lambda x: str(x).split("."))

In [35]:
elec = elec[["id","lyrics"]].drop(elec[elec.lyrics == "-99"].index)

In [39]:
elec.to_csv("elec_lyrics.csv", sep='\t', encoding='utf-8')


In [ ]:
#Lining up all the lyrics to create corpus
Corpus =''
for listitem in elec.lyrics:
    Corpus += listitem
    
Corpus = Corpus.lower() #converting all alphabets to lowecase 
print("Number of unique characters:", len(set(Corpus)))

Number of unique characters: 76


In [ ]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

The unique characters: ['\n', '\r', ' ', '!', '"', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '´', '¿', 'ß', 'à', 'á', 'ä', 'ç', 'è', 'é', 'í', 'ñ', 'ó', 'ö', 'ú', 'ü', 'œ', '–', '—', '’', '“', '”', '…']


In [ ]:
#Keeping only a limited set of characters. 
to_remove = ['{', '}', '~', '©', 'à', 'á', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ñ', 'ó', 'ö', 'ü', 'ŏ',
             'е', 'ا', 'س', 'ل', 'م', 'و', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '’', '‚', '“', '”', 
             '…', '\u205f', '\ufeff', '!', '&', '(', ')', '*', '-',  '/', ]
for symbol in to_remove:
    Corpus = Corpus.replace(symbol," ")

In [ ]:
print("The unique characters:",sorted(set(Corpus)))

The unique characters: ['\n', '\r', ' ', '"', "'", '+', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '´', '¿', 'ß', 'ú', 'œ']


In [ ]:
# Storing all the unique characters present in my corpus to bult a mapping dic. 
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

Total number of characters: 244181
Number of unique characters: 53


In [ ]:
#Splitting the Corpus in equal length of strings and output target
length = 40
features = []
targets = []
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
    
    
L_datapoints = len(targets)
print("Total number of sequences in the Corpus:", L_datapoints)

Total number of sequences in the Corpus: 244141


In [ ]:
f = open("text_electronic.txt", "a")
f.write(Corpus)
f.close()